In [0]:
from pyspark.sql.functions import from_unixtime, to_date, hour
from pyspark.sql.functions import col

In [0]:
city_dim_df = spark.read.parquet("dbfs:/tmp/city_dim.parquet")
date_dim_df = spark.table("date_dim")
time_dim_df = spark.table("time_dim")
weather_dim_df = spark.read.parquet("dbfs:/tmp/weather_dim.parquet")


In [0]:
weather_dim_df = weather_dim_df.withColumn("date", to_date(from_unixtime(col("dt"))))
weather_dim_df = weather_dim_df.withColumn("hour", hour(from_unixtime(col("dt"))))

In [0]:
joined_df = date_dim_df.join(weather_dim_df, date_dim_df["CalendarDate"] == weather_dim_df["date"], "inner")

In [0]:
# display(joined_df)

In [0]:
joined_df = joined_df.join(time_dim_df, time_dim_df["timeID"] == joined_df["hour"], "inner")


In [0]:
joined_df = joined_df.join(city_dim_df, joined_df["id"] == city_dim_df["id"], "inner")


In [0]:
# display(joined_df)

In [0]:
weather_data_hourly_df = joined_df.select(
    date_dim_df["dateKey"].alias("dateID"),
    time_dim_df["timeID"],
    city_dim_df["id"].alias("cityID"),
    weather_dim_df["weather_temp"],
    weather_dim_df["weather_description"],
    weather_dim_df["weather_id"],
    weather_dim_df["weather_main"],
    weather_dim_df["weather_icon"],
    weather_dim_df["base"],
    weather_dim_df["temp"],
    weather_dim_df["feels_like"],
    weather_dim_df["temp_min"],
    weather_dim_df["temp_max"],
    weather_dim_df["pressure"],
    weather_dim_df["humidity"],
    weather_dim_df["sea_level"],
    weather_dim_df["grnd_level"],
    weather_dim_df["visibility"],
    weather_dim_df["wind_speed"],
    weather_dim_df["wind_deg"],
    weather_dim_df["wind_gust"],
    weather_dim_df["clouds_all"],
    weather_dim_df["sunrise"],
    weather_dim_df["sunset"],
    weather_dim_df["cod"]
)


In [0]:
weather_data_hourly_df.write.mode("append").parquet("/tmp/weather_fact_hourly.parquet")